# Vintage Shop - Product Analysis

In [1]:
import requests
#BeautifulSoup es una calse dentro de la libreria bs4
from bs4 import BeautifulSoup
import pandas as pd
# from pprint import pprint
# from lxml import html
# from lxml.html import fromstring
# import urllib.request
from urllib.request import urlopen
# import random
import re
# import scrapy
import csv
import datetime

### Scrapping Aproach:

Fist, I've acces the main page and explore the configuration and structure.

I will be targeting all products, filtered as t-shirts

The page provides with a selection list to filter categories.I observed the same elements can be shown in more than one category. By analizing the products shown in in collection, we've notice that all items are at least in one the decades collections, and items that are in a decade selection are not in other decade, so by filtering by decade we will have each element in one specific category, with out dupplicates.

We want to retrive, the description, the selling price, the collection(decade) in with it is included and the link to the product especifications.

By analizing the html code I selected the main structure that shared all products, the "Product Card".

We will target all "Product Card" items in the same page and then iterate trough each of them to extract the required values.

Take in count that each collections has its products display in more than one page, and number of pages change between collections. We will need to know how many pages per colection there is in order to srap all products. 

Once we know who many pages there is in the same collection, we apply the next steps for each page:

1. Extract wanted values from each product of the page.
2. Set values to a DataFrame
3. Concatenate DataFrame pag(n) to the all_pages data frame

### 0. Analizing the url

By analizing the webpage and the sub-pages that we want to target.
Determine a fromated link that allow us to enter all pages by 
changing two variables; collection and page.

In [23]:
collection = "1980s"

In [24]:
page_no = 1 #works as tring and digit. we prefer digit for iteration in range (pages)

In [25]:
url = f"https://www.wycovintage.com/collections/{collection}?filter.p.product_type=Shirt&options%5Bprefix%5D=last&page={page_no}&sort_by=created-descending"

In [43]:
html = requests.get(url_90)
print(html)

<Response [200]>


In [39]:
soup = BeautifulSoup(html.content, "html.parser")

### 1. Explore collections:

In [2]:
url = "https://www.wycovintage.com/collections/all-products"
html = requests.get(url)
print(html)
soup = BeautifulSoup(html.content, "html.parser")

<Response [200]>


In [4]:
colections_tab = soup.find("select", {"name":"collection"})
collections_list = [i.getText().strip().replace(" ","-").lower() for i in colections_tab.find_all("option")]
#collections_list

In [5]:
exclude_colections = ["select-a-collection", "items-under-$200", 
    "displays",'all-products','backstage-passes','jackets',
    'jerseys','new-arrivals','shirts','shorts','skate','sweaters',
    'sweatshirts']

extract_colections = [i for i in collections_list if i not in exclude_colections]

### 2. Acces collections by pages:

In [336]:
def acces_collections(collection,page_no=1): 
    
    "This function generates a soup of the page 'x' of a filtered search based on collections"
    
    #1. We defin the url for the search based on collection and page:
    url = f"https://www.wycovintage.com/collections/{collection}?filter.p.product_type=Shirt&options%5Bprefix%5D=last&page={page_no}&sort_by=created-descending"

    #2. Reques the html code for the url mentioned
    html = requests.get(url)
    
    #3. Print the request resoponse, for visibility 
    print(html)
    
    #4. Return an objetct type BeautifulSoup with the content parsed
    return BeautifulSoup(html.content, "html.parser")

### 3. Extract nº of pages of a collection:

In [337]:
def last_page(soup):
    
    "This function returns the number of pages inside the same filtered shearch"
    
    try:
    
        #1. Find the element where pagination/index is contained
        index = soup.find("ul", {"class":"Pagination"})

        #2. Extract all elements inside pagination/index that represet a number page - list returned
        pags_list = index.find_all("a", {"class":"Button Pagination__Link"})

        #3. Print the last page, for visibility 
        print(f'Last page in pagination is nº{pags_list[-1].getText()}')

        #3. We acces the last element of the list = last page and retunr the text
        return int(pags_list[-1].getText())
    
    except AttributeError:
        print("There is just 1 page")
        return int(1)


### 4. Generate DF with all values from the same page:

In [338]:
def page_table(soup, collection):
    
    """Returns a DataFrame for a given soup"""
    
    #1. Fist we target the box elements where all the info of a product is contained:
    elemnt_box = soup.find_all("div", {"class":"ProductCard"})
    
    #2. We iterate trouth the elements to extract the info that we need: Comprehension lists
    """This work arround is not that straigth foward but we notice a bugg (see: debugging 1)"""
    
    #2.1 Descriptions list:
    descriptions = [ i.find("h2", {"class":"ProductCard__Title"}).getText().strip() for i in elemnt_box]
    
    #2.2 Prices list:
    prices = [ i.find("span", {"class":"Price"}).getText().strip() for i in elemnt_box]
    
    #2.3 Links:
    links = [ "https://www.wycovintage.com" + i.find("a").get("href") for i in elemnt_box]
    
    #3. Create a disctionary with the tree lists:
    page_dict = {}
    
    page_dict["description"] = descriptions
    page_dict["price"] = prices
    # There is an entry for each T-shit, even if its repeated -- will use it for counting later
    page_dict["qty"] = [1 for i in range(len(descriptions))]
    page_dict["collection"] = [collection for i in range(len(descriptions))]
    page_dict["link"] = links
    
    df = pd.DataFrame(page_dict)
    
    return df

### 6. Concatenate all pages in the same collection:

In [342]:
def all_pages_collection(collection):
    
    """Given a colection to target it extrats all selected values
        for all the pages in the collection and group them in a 
        unic dataframe """
     
    #1. Create an empty DataFrame (for agreggate date)
    df_t = pd.DataFrame()
    
    #2. Scrap the first page of the collection
    soup = acces_collections(collection, page_no=1)
    
    #3. Determine how many pages the collection have
    pages = last_page(soup)
    
    #4. Extract first page to a DataFrame
    df_t = page_table(soup, collection)
    pag = 1
    #   Print for visibility
    #print(f"First page was scraped from {collection}")
    
    # 5. If there is more than 1 page 
    if pages > 1:
        
        #Print for visibility
        print("Scraping more pages")
        
        #5.1 Extract all pages and concatenate to the initial dataframe(pag.1)
        
        for i in range(pages+1):#index starts with 0
            pag = int(i)+2 #first page was already sraped
            if pag <= pages:
                soup = acces_collections(collection,int(i)+2)
                df_g = page_table(soup, collection)
                df_t = pd.concat([df_t,df_g])
    
    print(f"{pag-2} pages were scraped form {collection}")
    return df_t

### 7. Export the dataframe generated to a .csv file


In [212]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"data/{file_name}_{t_stamp}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

## SCRAPING DATA - MAIN FUNCTION

We've analize the web to scrape and noticed that accesig collections by decades we can target all products in the page.

In [6]:
# Select the collections to srape:

col = ['1960s','1970s','1980s','1990s','2000s']

In [363]:
# main function:

for i in col:
    print(f'------ Scrapig {i} collection ------')
    df = all_pages_collection(i)
    export_to_csv(df, i)
    

------ Scrapig 1960s collection ------
<Response [200]>
There is just 1 page
-1 pages were scraped form 1960s
DataFrame successfully exported to data/1960s_2024-02-04_01:22:42.csv
------ Scrapig 1970s collection ------
<Response [200]>
Last page in pagination is nº12
Scraping more pages
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
12 pages were scraped form 1970s
DataFrame successfully exported to data/1970s_2024-02-04_01:23:00.csv
------ Scrapig 1980s collection ------
<Response [200]>
Last page in pagination is nº36
Scraping more pages
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Respons